In [ ]:
!git clone https://github.com/state-spaces/mamba.git

fatal: destination path 'mamba' already exists and is not an empty directory.


In [ ]:
%cd mamba
!ls

/content/mamba
assets	 benchmarks  evals    mamba_ssm       README.md   setup.py
AUTHORS  csrc	     LICENSE  pyproject.toml  rocm_patch  tests


In [ ]:
# !pip install torch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 --index-url https://download.pytorch.org/whl/cu118
# !pip install causal_conv1d==1.1.1
# !pip install mamba-ssm==1.2.0.post1

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


  Using cached mamba_ssm-1.2.0.post1.tar.gz (34 kB)
  Preparing metadata (setup.py) ... done
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/us

In [ ]:
!pip install https://github.com/Dao-AILab/causal-conv1d/releases/download/v1.4.0/causal_conv1d-1.4.0+cu118torch2.0cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
!pip install mamba-ssm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.6/103.6 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: causal-conv1d
    Found existing installation: causal-conv1d 1.1.1
    Uninstalling causal-conv1d-1.1.1:
      Successfully uninstalled causal-conv1d-1.1.1
  Using cached mamba_ssm-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=343412612 sha256=150be68a91b3444b35ee2cda6a4790aaf83711009a8bf8e91afc7a316b091df6
  Stored in directory: /root/.cache/pip/wheels/57/7c/90/9f963468ecc3791e36e388f9e7b4a4e1e3f90fbb340055aa4d
Successfully built mamba-ssm


Below is the given code to create a Mamba block

In [ ]:
import torch
from mamba_ssm import Mamba
device = "cuda" if torch.cuda.is_available() else "cpu"
batch, length, dim = 2, 64, 16
x = torch.randn(batch, length, dim).to(device)
model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=16,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to(device)
y = model(x)
assert y.shape == x.shape
print(model)

Mamba(
  (in_proj): Linear(in_features=16, out_features=64, bias=False)
  (conv1d): Conv1d(32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
  (act): SiLU()
  (x_proj): Linear(in_features=32, out_features=33, bias=False)
  (dt_proj): Linear(in_features=1, out_features=32, bias=True)
  (out_proj): Linear(in_features=32, out_features=16, bias=False)
)


In [ ]:
!pip install ptflops

Here we create a Mamba block and train it on MNIST dataset, after which we test the performance of pruning on it.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from mamba_ssm import Mamba

device = "cuda" if torch.cuda.is_available() else "cpu"

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

dim = 16
model = Mamba(
    d_model=dim,
    d_state=16,
    d_conv=4,
    expand=2
).to(device)

def train_model(model, train_loader, optimizer, criterion, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for images, labels in train_loader:
            images = images.view(images.size(0), -1, dim).to(device)
            labels = labels.to(device)

            outputs = model(images)
            outputs = outputs.mean(dim=1)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(images.size(0), -1, dim).to(device)
            labels = labels.to(device)

            outputs = model(images)
            outputs = outputs.mean(dim=1)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train_model(model, train_loader, optimizer, criterion, num_epochs=5)
evaluate_model(model, test_loader)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 43270756.59it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1186441.66it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 10960295.08it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3795682.16it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Loss: 1.7688
Epoch [2/5], Loss: 0.9396
Epoch [3/5], Loss: 0.7102
Epoch [4/5], Loss: 0.6189
Epoch [5/5], Loss: 0.5659
Test Accuracy: 83.12%


In [ ]:
torch.save(model, "original_model_complete.pth")

In [ ]:
print(model)

Mamba(
  (in_proj): Linear(in_features=16, out_features=64, bias=False)
  (conv1d): Conv1d(32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
  (act): SiLU()
  (x_proj): Linear(in_features=32, out_features=33, bias=False)
  (dt_proj): Linear(in_features=1, out_features=32, bias=True)
  (out_proj): Linear(in_features=32, out_features=16, bias=False)
)


In [ ]:
from ptflops import get_model_complexity_info
import torch.nn.utils.prune as prune

def print_model_complexity(model, input_tensor):
    macs, params = get_model_complexity_info(model, input_tensor.shape, as_strings=True, print_per_layer_stat=True)
    print(f"FLOPs: {macs}, Parameters: {params}")


def prune_model(model, pruning_amount=0.1):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            prune.l1_unstructured(module, name='weight', amount=pruning_amount)
            print(f"Pruned Conv1d layer: {name}")
        elif isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=pruning_amount)
            print(f"Pruned Linear layer: {name}")

    return model

def structured_pruning(model, pruning_amount=0.1):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            prune.ln_structured(module, name='weight', amount=pruning_amount, n=2, dim=1)
            print(f"Pruned Conv2d layer: {name}")
        elif isinstance(module, nn.Linear):
            prune.ln_structured(module, name='weight', amount=pruning_amount, n=2, dim=0)
            print(f"Pruned Linear layer: {name}")
    return model

model_copy = torch.load("original_model_complete.pth")

for images, labels in train_loader:
  input_data = images.view(images.size(0), -1, dim).to(device)
  break
input_data = torch.randn(64, 1, 28, 28).to(device)
print("Before pruning:")

flops, params = get_model_complexity_info(model_copy.conv1d, (32,32),as_strings=True,print_per_layer_stat=True)

pmodel = structured_pruning(model_copy, pruning_amount=0.3)

print("After pruning:")
print(model_copy.conv1d)
print(pmodel.conv1d)
pflops, pparams = get_model_complexity_info(pmodel.conv1d, (32,32),as_strings=True,print_per_layer_stat=True)

evaluate_model(pmodel, test_loader)


Before pruning:
Conv1d(160, 100.000% Params, 5.6 KMac, 100.000% MACs, 32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
Pruned Linear layer: in_proj
Pruned Conv2d layer: conv1d
Pruned Linear layer: x_proj
Pruned Linear layer: dt_proj
Pruned Linear layer: out_proj
After pruning:
Conv1d(32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
Conv1d(32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
Conv1d(160, 100.000% Params, 5.6 KMac, 100.000% MACs, 32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
Test Accuracy: 11.33%
